In [52]:
DOWNLOAD_LATEST = False
import os
if DOWNLOAD_LATEST:
    env_vars = open(".env.development").read()
    DB_CONNECTION_STRING = env_vars.split("DB_CONNECTION_STRING = ")[1].split("?authSource")[0]
    os.system(f"mongoexport --uri {DB_CONNECTION_STRING} --collection users --out data/Curio/users.json")
    os.system(f"mongoexport --uri {DB_CONNECTION_STRING} --collection posts --out data/Curio/posts.json")
    os.system(f"mongoexport --uri {DB_CONNECTION_STRING} --collection groups --out data/Curio/groups.json")
    os.system(f"mongoexport --uri {DB_CONNECTION_STRING} --collection reactions --out data/Curio/reactions.json")
    os.system(f"mongoexport --uri {DB_CONNECTION_STRING} --collection echopostproposals --out data/Curio/echopostproposals.json")
    os.system(f"mongoexport --uri {DB_CONNECTION_STRING} --collection viewtimestamps --out data/Curio/viewtimestamps.json")

In [53]:
import jsonlines
import pandas as pd
from superdebug import debug
from collections import defaultdict
curio_data_path = "data/Curio/"
users = jsonlines.Reader(open(curio_data_path + "users.json"))
posts = jsonlines.Reader(open(curio_data_path + "posts.json"))
groups = jsonlines.Reader(open(curio_data_path + "groups.json"))
reactions = jsonlines.Reader(open(curio_data_path + "reactions.json"))
echopostproposals = jsonlines.Reader(open(curio_data_path + "echopostproposals.json"))
viewtimestamps = jsonlines.Reader(open(curio_data_path + "viewtimestamps.json"))


##### Record necessary information: `group_id_name_map, group_id_members_id_map, user_id_name_map, user_id_group_id_viewtime_map`

In [54]:
user_id_name_map = {}
for user in users:
    user_id = user['_id']["$oid"]
    user_name = user["username"]
    user_id_name_map[user_id] = user_name
    
group_id_members_id_map = {}
group_id_members_name_map = {}
group_name_member_num_map = {}
group_id_name_map = {}
for group in groups:
    group_id = group['_id']["$oid"]
    group_name = group["name"].replace(" ", "|")
    group_id_name_map[group_id] = group_name
    members = [_["$oid"] for _ in group['members']]
    group_id_members_id_map[group_id] = members
    group_id_members_name_map[group_name] = [user_id_name_map[user_id] for user_id in members if user_id in user_id_name_map]
    if "TEST" not in group_name:
        group_name_member_num_map[group_name] = len(members)
debug(group_member_num = group_name_member_num_map)
print("group_id_members_name_map:", group_id_members_name_map)
    
user_id_group_id_viewtime_map = defaultdict(dict)
for viewtimestamp in viewtimestamps:
    if viewtimestamp["user"] is None: continue
    user_id = viewtimestamp["user"]["$oid"]
    if "group" not in viewtimestamp or viewtimestamp["group"] == None:
        group_ids = []
        for group_id in group_id_members_id_map:
            if user_id in group_id_members_id_map[group_id]:
                group_ids.append(group_id)
    else:
        group_ids = [viewtimestamp["group"]["$oid"]]
    dt_obj = pd.to_datetime(viewtimestamp["timestamp"]["$date"], infer_datetime_format=True)
    timestamp = dt_obj.timestamp()
    for group_id in group_ids:
        if group_id not in user_id_group_id_viewtime_map[user_id] or timestamp > user_id_group_id_viewtime_map[user_id][group_id]:
            user_id_group_id_viewtime_map[user_id][group_id] = timestamp
    

------------------ 2022-11-01 08:32:22 ------------------
DEBUG: 1 vars: ['group_member_num'], at <ipython-input-54-5d1bfe551d68>:20 <module>
0 / 26.  group_member_num dict {...} with 14 keys ['LOLCHI', 'Trivia', 'Team|Llama', 'Team|Mignon', 'The|Positive|Corner', 'Random', 'Food', 'Out|of|Context|Quotes', 'Team|PhThree', 'Stanford|HCI', 'STANFORD', 'Campus|Events', 'Stanford|Wordle', 'Where|Am|I']
    LOLCHI num val: 23
    Trivia num val: 18
    Team|Llama num val: 7
    Team|Mignon num val: 9
    The|Positive|Corner num val: 16
    Random num val: 17
    Food num val: 21
    Out|of|Context|Quotes num val: 16
    Team|PhThree num val: 7
    Stanford|HCI num val: 22
    STANFORD num val: 4
    Campus|Events num val: 5
    Stanford|Wordle num val: 2
    Where|Am|I num val: 6
------------------ 2022-11-01 08:32:22 ------------------
group_id_members_name_map: {'Parent|(TEST)': ['parth', 'msb', 'parnoud', 'kjeong', 'msbtest4', 'Gayoung', 'jigar'], 'LOLCHI': ['parth', 'msb', 'wanrong', 'k

##### Aggregate information about each post, and compose `posts_df`

In [55]:
import time
from collections import defaultdict
posts_info_map = defaultdict(dict)
group_name_post_num_map = defaultdict(int)
group_name_first_post_time_map = defaultdict(int)
group_name_active_member_map = defaultdict(set)
for post in posts:
    group_id = post["group"]["$oid"]
    group_name = group_id_name_map[group_id]
    if "TEST" in group_name or "test" in group_name:
        continue
    
    group_name_post_num_map[group_name] += 1
    
    post_id = post["_id"]["$oid"]
    post_info = posts_info_map[post_id] # information about the post
    post_info["SUBMISSION_ID"] = post_id
    
    post_info["SUBREDDIT"] = group_name
    post_info["group_id"] = group_id
    title = post["contents"]
    post_info["TITLE"] = title
    
    author_id = post["author"]["$oid"]
    author = user_id_name_map[author_id]
    post_info["author_id"] = author_id
    post_info["AUTHOR"] = author
    post_info["NSFW"] = None
    post_info["LINK"] = ""
    
    dt_obj = pd.to_datetime(post["updatedAt"]["$date"], infer_datetime_format=True)
    timestamp = dt_obj.timestamp()
    post_info["timestamp"] = timestamp
    if (group_name not in group_name_first_post_time_map) or timestamp < group_name_first_post_time_map[group_name]:
        group_name_first_post_time_map[group_name] = timestamp
    
    if abs(time.time() - timestamp) <= 10 * 24 * 60 * 60 and author_id in group_id_members_id_map[group_id]:
        group_name_active_member_map[group_name].add(author_id)
        
    
    if "#_COMMENTS" not in post_info:
        post_info["#_COMMENTS"] = 0
        
    if "threadParent" in post:
        threadParent_id = post["threadParent"]["$oid"]
        if "#_COMMENTS" not in posts_info_map[threadParent_id]:
            posts_info_map[threadParent_id]["#_COMMENTS"] = 1
        else:
            posts_info_map[threadParent_id]["#_COMMENTS"] += 1


Analyze statistics

In [56]:
group_name_post_frequency_map = {group_name: (60 * 60 * 24 * group_name_post_num_map[group_name])/(time.time() - group_name_first_post_time_map[group_name]) for group_name in group_name_post_num_map}
print({group_name: f"{group_name_post_frequency_map[group_name]:.4f}" for group_name in group_name_post_frequency_map})
debug(group_posting_frequency_day = {group_name: f"{group_name_post_frequency_map[group_name]:.4f}" for group_name in group_name_post_frequency_map})

group_name_post_frequency_per_person_map = {group_name: group_name_post_frequency_map[group_name]/group_name_member_num_map[group_name] for group_name in group_name_post_frequency_map}
print({group_name: f"{group_name_post_frequency_per_person_map[group_name]:.4f}" for group_name in group_name_post_frequency_per_person_map})
debug(group_posting_frequency_per_person_day = {group_name: f"{group_name_post_frequency_per_person_map[group_name]:.4f}" for group_name in group_name_post_frequency_per_person_map})
posts_df = pd.DataFrame.from_records(list(posts_info_map.values()))

{'LOLCHI': '0.1694', 'Trivia': '0.4017', 'Random': '0.2094', 'Team|Llama': '0.6663', 'Team|Mignon': '0.8509', 'The|Positive|Corner': '0.5072', 'Food': '0.2767', 'Out|of|Context|Quotes': '0.5279', 'Team|PhThree': '0.1451', 'Stanford|HCI': '0.0484', 'Campus|Events': '1.0731', 'Where|Am|I': '1.3396', 'STANFORD': '0.1342'}
------------------ 2022-11-01 08:32:23 ------------------
DEBUG: 1 vars: ['group_posting_frequency_day'], at <ipython-input-56-43d8415bbca7>:3 <module>
0 / 27.  group_posting_frequency_day dict {...} with 13 keys ['LOLCHI', 'Trivia', 'Random', 'Team|Llama', 'Team|Mignon', 'The|Positive|Corner', 'Food', 'Out|of|Context|Quotes', 'Team|PhThree', 'Stanford|HCI', 'Campus|Events', 'Where|Am|I', 'STANFORD']
    LOLCHI str len 6: 0.1694
    Trivia str len 6: 0.4017
    Random str len 6: 0.2094
    Team|Llama str len 6: 0.6663
    Team|Mignon str len 6: 0.8509
    The|Positive|Corner str len 6: 0.5072
    Food str len 6: 0.2767
    Out|of|Context|Quotes str len 6: 0.5279
    Team

##### Record users' votes on posts and compose `votes_df`

In [57]:
posts_users_votes = defaultdict(dict)
for post_id, post_info in posts_info_map.items():
    if "SUBMISSION_ID" not in post_info or "SUBREDDIT" not in post_info: continue
    if post_info["SUBREDDIT"] == "" or post_info["SUBREDDIT"] == None: continue
    assert post_id == post_info["SUBMISSION_ID"]
    post_timestamp = post_info["timestamp"]
    group_id = post_info["group_id"]
    author_id = post_info["author_id"]
    group_name = post_info["SUBREDDIT"]
    group_members = group_id_members_id_map[group_id]
    
    # fill in downvotes
    users_votes = posts_users_votes[post_id]
    for member_id in group_members:
        if member_id in user_id_name_map and (group_id not in user_id_group_id_viewtime_map[member_id] or post_timestamp < user_id_group_id_viewtime_map[member_id][group_id]): # TODO:
            users_votes[member_id] = {"SUBMISSION_ID": post_id,	"SUBREDDIT": "r/" + group_name, "CREATED_TIME": post_timestamp, "USERNAME": user_id_name_map[member_id], "VOTE": "downvote"}
    if author_id in users_votes:
        users_votes[author_id]["VOTE"] = "upvote" # author always upvotes
    else:
        users_votes[author_id] = {"SUBMISSION_ID": post_id,	"SUBREDDIT": "r/" + group_name, "CREATED_TIME": post_timestamp, "USERNAME": user_id_name_map[member_id], "VOTE": "upvote"}
        

In [58]:
for reaction in reactions:
    post_id = reaction["post"]["$oid"]
    if post_id not in posts_users_votes: continue
    author_id = reaction["author"]["$oid"]
    if author_id in posts_users_votes[post_id]:
        posts_users_votes[post_id][author_id]["VOTE"] = "upvote"
    else:
        posts_users_votes[post_id][author_id] = list(posts_users_votes[post_id].values())[0]
        posts_users_votes[post_id][author_id]["VOTE"] = "upvote"
        posts_users_votes[post_id][author_id]["USERNAME"] = user_id_name_map[author_id]
        
    group_id = posts_info_map[post_id]["group_id"]
    group_name = group_id_name_map[group_id]
    dt_obj = pd.to_datetime(reaction["updatedAt"]["$date"], infer_datetime_format=True)
    timestamp = dt_obj.timestamp()
    if abs(time.time() - timestamp) <= 10 * 24 * 60 * 60 and author_id in group_id_members_id_map[group_id]:
        group_name_active_member_map[group_name].add(author_id)

for echopostproposal in echopostproposals:
    post_id = echopostproposal["echoParent"]["$oid"]
    if post_id not in posts_users_votes: continue
    author_id = echopostproposal["echoAuthor"]["$oid"]
    if author_id not in posts_users_votes[post_id]:
        posts_users_votes[post_id][author_id] = list(posts_users_votes[post_id].values())[0]
        posts_users_votes[post_id][author_id]["VOTE"] = "upvote"
        posts_users_votes[post_id][author_id]["USERNAME"] = user_id_name_map[author_id]
    else:
        posts_users_votes[post_id][author_id]["VOTE"] = "upvote"
        
    group_id = posts_info_map[post_id]["group_id"]
    group_name = group_id_name_map[group_id]
    
    post_timestamp = posts_info_map[post_id]["timestamp"]
    if abs(time.time() - post_timestamp) <= 10 * 24 * 60 * 60 and author_id in group_id_members_id_map[group_id]:
        group_name_active_member_map[group_name].add(author_id)

        


In [59]:
all_votes = []
group_name_upvote_num_map = defaultdict(int)
for post_id, users_votes in posts_users_votes.items():
    for user_id, vote in users_votes.items():
        all_votes.append(vote)
        if vote["VOTE"] == "upvote":
            group_name_upvote_num_map[vote["SUBREDDIT"][2:]] += 1
# group_name_upvote_frequency_map = {group_name: f"{(time.time() - group_name_first_post_time_map[group_name])/(60 * 60 * 24 * group_name_upvote_num_map[group_name]):.4f}" for group_name in group_name_upvote_num_map}
# debug(group_upvoteing_frequency_day = group_name_upvote_frequency_map)

group_name_upvote_frequency_map = {group_name: (60 * 60 * 24 * group_name_upvote_num_map[group_name])/(time.time() - group_name_first_post_time_map[group_name]) for group_name in group_name_upvote_num_map}
print({group_name: f"{group_name_upvote_frequency_map[group_name]:.4f}" for group_name in group_name_upvote_frequency_map})
debug(group_posting_frequency_day = {group_name: f"{group_name_upvote_frequency_map[group_name]:.4f}" for group_name in group_name_upvote_frequency_map})
debug(group_active_member_num = str({name: len(members) for name, members in group_name_active_member_map.items()}))
votes_df = pd.DataFrame.from_records(all_votes)

{'LOLCHI': '0.4488', 'Trivia': '0.5834', 'Random': '0.5330', 'Team|Llama': '0.7805', 'Team|Mignon': '1.0230', 'The|Positive|Corner': '1.2345', 'Food': '0.7379', 'Out|of|Context|Quotes': '1.5941', 'Team|PhThree': '0.1866', 'Stanford|HCI': '0.1937', 'Campus|Events': '1.8779', 'Where|Am|I': '1.4736', 'STANFORD': '0.1342'}
------------------ 2022-11-01 08:32:24 ------------------
DEBUG: 1 vars: ['group_posting_frequency_day'], at <ipython-input-59-e9cd0c29aa30>:13 <module>
0 / 29.  group_posting_frequency_day dict {...} with 13 keys ['LOLCHI', 'Trivia', 'Random', 'Team|Llama', 'Team|Mignon', 'The|Positive|Corner', 'Food', 'Out|of|Context|Quotes', 'Team|PhThree', 'Stanford|HCI', 'Campus|Events', 'Where|Am|I', 'STANFORD']
    LOLCHI str len 6: 0.4488
    Trivia str len 6: 0.5834
    Random str len 6: 0.5330
    Team|Llama str len 6: 0.7805
    Team|Mignon str len 6: 1.0230
    The|Positive|Corner str len 6: 1.2345
    Food str len 6: 0.7379
    Out|of|Context|Quotes str len 6: 1.5941
    Tea

In [60]:
group_name_upvote_rate_map = {group_name: (group_name_upvote_num_map[group_name])/(group_name_post_num_map[group_name]) for group_name in group_name_upvote_num_map}
print({group_name: f"{group_name_upvote_rate_map[group_name]:.4f}" for group_name in group_name_upvote_rate_map})
group_name_upvote_rate_per_person_map = {group_name: (group_name_upvote_num_map[group_name])/(group_name_post_num_map[group_name] * group_name_member_num_map[group_name]) for group_name in group_name_upvote_num_map}
print({group_name: f"{group_name_upvote_rate_per_person_map[group_name]:.4f}" for group_name in group_name_upvote_rate_per_person_map})

{'LOLCHI': '2.6500', 'Trivia': '1.4524', 'Random': '2.5455', 'Team|Llama': '1.1714', 'Team|Mignon': '1.2022', 'The|Positive|Corner': '2.4340', 'Food': '2.6667', 'Out|of|Context|Quotes': '3.0196', 'Team|PhThree': '1.2857', 'Stanford|HCI': '4.0000', 'Campus|Events': '1.7500', 'Where|Am|I': '1.1000', 'STANFORD': '1.0000'}
{'LOLCHI': '0.1152', 'Trivia': '0.0807', 'Random': '0.1497', 'Team|Llama': '0.1673', 'Team|Mignon': '0.1336', 'The|Positive|Corner': '0.1521', 'Food': '0.1270', 'Out|of|Context|Quotes': '0.1887', 'Team|PhThree': '0.1837', 'Stanford|HCI': '0.1818', 'Campus|Events': '0.3500', 'Where|Am|I': '0.1833', 'STANFORD': '0.2500'}


In [61]:
posts_df.to_csv(curio_data_path + "post_info.txt", sep = "\t", index = False)
votes_df.to_csv(curio_data_path + "all_votes.txt", sep = "\t", index = False)